In [1]:
from nltk.tokenize import sent_tokenize
from collections import Counter
import string

In [2]:
with open('alice_wonderland.txt', 'r', encoding='utf-8') as fh:
    alltext = fh.read()


In [3]:
cleaned_text = alltext.lower()
cleaned_text = cleaned_text.translate({ord('\n'):' ', 
                                  ord('\"'):' ', 
                                  ord('’'):' ',
                                  ord('‘'):' ',    
                                  ord('\''):' '
                                 })

In [4]:
sents = sent_tokenize(cleaned_text)

In [5]:
# 1. replace punctuations with spaces
# 2. add sentence end markers
translator = str.maketrans(string.punctuation, ' '*len(string.punctuation))
sents = [x for x in map(lambda x: x.translate(translator).strip(), sents)]
sents = [x for x in map(lambda x: x + ' </s>', sents)]

In [6]:
corpus = []
for sent in sents:
    thiswords = [x for x in filter(lambda x: len(x)>0, sent.split(' '))]
    corpus.extend(thiswords)

In [7]:
class NGramModel():
    def __init__(self, n, corpus):
        self.n = n
        self.corpus = corpus
        self.counter = None
        # index zero is n-1 gram, index one is n-2 gram etc.
        self.subModels = []
        
    def buildModels(self):
        self.ngrams = []
        self.proba_matrix = {}
        
        """
        to build probabilities of ngram model, we also need to count
        n-1 gram, n-2 gram etc all the way down to unigram
        """
        for sub_n in range(self.n-1, 0, -1):
            n_minus_model = NGramModel(sub_n, self.corpus)
            # this is a weird recursive call
            n_minus_model.buildModels()
            self.subModels.append(n_minus_model)
        
        # build the actual model...
        for i in range(len(self.corpus)-(self.n-1)):
            this_tuple = []
            for tindx in range(i, i+self.n):
                this_tuple.append(self.corpus[tindx])
            self.ngrams.append(tuple(this_tuple))
            
            
        self.counter = Counter(self.ngrams)
        
        # now build probability matrices
        
        if self.n > 1:
            for tup in self.counter:
                given_ngram = tup[:-1]
                new_word = tup[-1]
                
                # init the dict
                if given_ngram not in self.proba_matrix:
                    self.proba_matrix[given_ngram] = {}
                
                denominator = self.subModels[0].get_count(given_ngram)
                numerator = self.get_count(tup)
                proba = numerator / denominator
                self.proba_matrix[given_ngram][new_word] = proba
    
    
    
    def most_common(self, n=15):
        return self.counter.most_common(n)
    
    def get_count(self, words):
        cc = self.counter.get(words)
        return cc if cc else 0            
    
    def get_proba(self, sentence):
        # assume ngram is correct length, i.e self.n
        ngram = tuple(sentence.split(' '))
        
        if len(ngram) == self.n:
            given_phrase = ngram[:-1]
            last_word = ngram[-1]
            return self.proba_matrix[given_phrase][last_word]
        else:
            raise ValueError("wrong length of sentence")

In [8]:
m1 = NGramModel(2,corpus)
m1.buildModels()

In [9]:
m1.get_proba('the dormouse')

0.02130249543517955

In [10]:
m1.get_proba('the door')

0.01278149726110773

In [11]:
for k in range(10,20):
    t1 = m1.most_common(k)[-1][0]
    dd = m1.subModels[0].get_count(t1[:-1]) # denominator
    nn = m1.get_count(t1) # numerator
    print ("The count of {} in {} gram model is {}".format(t1,m1.n,nn))
    print ("The count of {} in {} gram model is {}".format(t1[:-1], m1.n-1, dd))
    print ("The probability is thus {}\n".format(nn/dd))

The count of ('in', 'the') in 2 gram model is 79
The count of ('in',) in 1 gram model is 369
The probability is thus 0.2140921409214092

The count of ('it', 'was') in 2 gram model is 76
The count of ('it',) in 1 gram model is 593
The probability is thus 0.1281618887015177

The count of ('the', 'queen') in 2 gram model is 72
The count of ('the',) in 1 gram model is 1643
The probability is thus 0.04382227632379793

The count of ('to', 'the') in 2 gram model is 69
The count of ('to',) in 1 gram model is 729
The probability is thus 0.09465020576131687

The count of ('</s>', 'and') in 2 gram model is 68
The count of ('</s>',) in 1 gram model is 1599
The probability is thus 0.04252657911194497

The count of ('the', 'king') in 2 gram model is 62
The count of ('the',) in 1 gram model is 1643
The probability is thus 0.03773584905660377

The count of ('as', 'she') in 2 gram model is 61
The count of ('as',) in 1 gram model is 263
The probability is thus 0.23193916349809887

The count of ('don', '

In [12]:
for k in m1.subModels[0].counter:
    print (k)

('brain',)
('accident',)
('want',)
('spell',)
('dreaming',)
('hurriedly',)
('break',)
('follows',)
('“french',)
('bent',)
('advantage',)
('jar',)
('“william',)
('shaking',)
('dare',)
('editions',)
('shoulder',)
('saucer',)
('because',)
('your',)
('last',)
('difficulties',)
('m',)
('unless',)
('leaders',)
('locked',)
('choosing',)
('examining',)
('broken',)
('twentieth',)
('delighted',)
('sprawling',)
('“up',)
('spite',)
('stupidly',)
('treacle',)
('tart',)
('ink',)
('knave',)
('returning',)
('neck',)
('missed',)
('precious',)
('seem',)
('grunted',)
('before',)
('balls',)
('hour',)
('immediate',)
('breeze',)
('milk',)
('sadly',)
('kindly',)
('about',)
('lamps',)
('invented',)
('squeezed',)
('corners',)
('coils',)
('rich',)
('instantly',)
('remarked',)
('sweet',)
('legged',)
('tide',)
('promise',)
('ridges',)
('executioner',)
('half',)
('shiny',)
('mistake',)
('kill',)
('messages',)
('series',)
('scratching',)
('begun',)
('window',)
('worm',)
('explain',)
('twelfth',)
('proper',)
('rest'

('encouraging',)
('fell',)
('melancholy',)
('guess',)
('royal',)
('interrupted',)
('majesty',)
('conversations',)
('fact',)
('ring',)
('ate',)
('quarrelling',)
('lay',)
('can',)
('eaglet',)
('immense',)
('curls',)
('he',)
('gryphon',)
('shade',)
('dinner',)
('procession',)
('finding',)
('pronounced',)
('readily',)
('jumped',)
('worse',)
('must',)
('assembled',)
('prove',)
('pine',)
('get”',)
('choking',)
('fallen',)
('pet',)
('patriotic',)
('vote',)
('softly',)
('believe',)
('written',)
('dance',)
('meaning',)
('nobody',)
('leaves',)
('“never',)
('idiotic',)
('o',)
('interrupting',)
('bring',)
('“coming',)
('high',)
('tell',)
('mushroom',)
('cakes',)
('“we',)
('long',)
('means',)
('bones',)
('trusts',)
('grand',)
('rustling',)
('paused',)
('passionate',)
('crumbs',)
('prize',)
('stirring',)
('“oh',)
('bag',)
('doesn',)
('pencils',)
('when',)
('orange',)
('desperately',)
('answers',)
('dressed',)
('party',)
('towards',)
('prevent',)
('jury',)
('chapter',)
('pretexts',)
('rises',)
('beco